# HW6 Comprehensive Supervised Learning
Contributors: Zongyu Wu, Tony Wilson, Chandler Smith

Summary: The overall purpose of this assignment is to tie together a variety of supervised learning techniques in order to appropriately analyze several questions related to the Behavioral Risk Factor Surveillance System. More specifically, the goal is to use supervised ML to identify patterns of comorbidity among the survey respondents. This is a comparative exercise focusing on pre (2019) and post (2021) covid health. 

Notes from discussion:

Approach: One approach for understanding behavioral factors behind comorbidity is to apply classification algorithms to the BRFSS data. Such algorithms can be trained to predict whether an individual has multiple chronic conditions based on their responses to the survey questions. By analyzing the features that are most important for predicting comorbidity, one can identify risk factors and inform the development of targeted prevention and intervention strategies.

Response related:
Depression = ADDEPEV3
 Ever told Asthma = _CASTHM1
 COPD = (CHCCOPD2 for 2019 and CHCCOPD3 for 2021)
 Cancer = (combine CHCSCNCR and CHCOCNCR)
 Ever told Heart Condition = combination of CVDCRHD4, CVDINFR4 and CVDSTRK3
 Diabetes = DIABETE4
 

Key demographic features:
Age: _AGE_G
 Marital: MARITAL
 Sex: _SEX
 Income: INCOME2
 Education: _EDUCAG

- Do NOT refer to correlation
- BRFSS data can be used to understand comorbidity, which refers to the presence of multiple chronic conditions in an individual. 
- RFSS uses a complex sampling and weighting scheme to measure prevalence of many health conditions, behavioral and lifestyle related risk factors and emerging health issues in states. 
- Do not use the weight variable as we are conducting estimations. 


- 2019 Codebook: https://www.cdc.gov/brfss/annual_data/2019/pdf/codebook19_llcp-v2-508.HTML
- 2021 Codebook: https://www.cdc.gov/brfss/annual_data/2021/pdf/codebook21_llcp-v2-508.pdf

## Clean, Standardize, and Merge data


In [36]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

import sklearn 

# Libraries related to outlier detection
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope

from sklearn import datasets
import random
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings('ignore') 
sns.set(rc={'figure.figsize':(11,8)})

pd.options.display.float_format = '{:.2f}'.format

## <font color= darkgreen> Basic EDA

### <font color= lightblue> Step 1: Read the data and merge into Pandas Data Frame

In [37]:
###############################################################################################################################################################################
'''                                                                       Data Frame Setup                                                                                  '''
###############################################################################################################################################################################
## The first column is index: skipping that column to end read csv to Panda Data Frame
df_21 = pd.read_csv("res\\brfss21-1.csv")
df_21.drop(columns="Unnamed: 0", inplace=True)
df_21.head()



,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_FRTRES1,_VEGRES1,_FRUTSU1,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1
0,1.00,1.00,b'01192021',b'01',b'19',b'2021',1100.00,b'2021000001',2021000001.00,1.00,...,1.00,1.00,100.00,214.00,1.00,1.00,1.00,1.00,0.00,0.00
1,1.00,1.00,b'01212021',b'01',b'21',b'2021',1100.00,b'2021000002',2021000002.00,1.00,...,1.00,1.00,100.00,128.00,1.00,1.00,1.00,1.00,0.00,0.00
2,1.00,1.00,b'01212021',b'01',b'21',b'2021',1100.00,b'2021000003',2021000003.00,1.00,...,1.00,1.00,100.00,71.00,1.00,2.00,1.00,1.00,0.00,0.00
3,1.00,1.00,b'01172021',b'01',b'17',b'2021',1100.00,b'2021000004',2021000004.00,1.00,...,1.00,1.00,114.00,165.00,1.00,1.00,1.00,1.00,0.00,0.00
4,1.00,1.00,b'01152021',b'01',b'15',b'2021',1100.00,b'2021000005',2021000005.00,1.00,...,1.00,1.00,100.00,258.00,1.00,1.00,1.00,1.00,0.00,0.00


In [38]:
df_19 = pd.read_csv("res\\brfss19-1.csv")
df_19.drop(columns="Unnamed: 0", inplace= True)
df_19.head()


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1,1,1182019,1,18,2019,1100,2019000001,2019000001,1.00,...,114.00,1,1,1,1,0,0,2.00,1.00,2.00
1,1,1,1132019,1,13,2019,1100,2019000002,2019000002,1.00,...,121.00,1,1,1,1,0,0,1.00,1.00,2.00
2,1,1,1182019,1,18,2019,1100,2019000003,2019000003,1.00,...,164.00,1,1,1,1,0,0,1.00,2.00,2.00
3,1,1,1182019,1,18,2019,1200,2019000004,2019000004,1.00,...,NaN,9,9,1,1,1,1,9.00,9.00,NaN
4,1,1,1042019,1,4,2019,1100,2019000005,2019000005,1.00,...,178.00,1,1,1,1,0,0,2.00,1.00,2.00


### <font color= lightblue> Step 2: Summary of Stats

In [39]:
###############################################################################################################################################################################
'''                                                                    Data Frame Description                                                                               '''
###############################################################################################################################################################################
df_19.describe()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1,_FLSHOT7,_PNEUMO3,_AIDTST4
count,418268.00,418268.00,418268.00,418268.00,418268.00,418268.00,418268.00,418268.00,418268.00,149941.00,...,364838.00,418268.00,418268.00,418268.00,418268.00,418268.00,418268.00,159112.00,159112.00,377977.00
mean,29.62,6.54,6727352.51,6.58,14.53,2019.04,1117.44,2019004884.52,2019004884.52,1.00,...,204.14,2.19,2.19,1.00,1.00,0.11,0.13,2.23,2.37,1.97
std,16.15,3.34,3304672.99,3.31,8.49,0.21,37.95,3653.32,3653.32,0.00,...,267.90,2.40,2.63,0.04,0.05,0.32,0.35,2.47,2.73,1.56
min,1.00,1.00,1012020.00,1.00,1.00,2019.00,1100.00,2019000001.00,2019000001.00,1.00,...,0.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00
25%,18.00,4.00,4082019.00,4.00,7.00,2019.00,1100.00,2019002011.00,2019002011.00,1.00,...,114.00,1.00,1.00,1.00,1.00,0.00,0.00,1.00,1.00,1.00
50%,27.00,7.00,7012019.00,7.00,14.00,2019.00,1100.00,2019004137.00,2019004137.00,1.00,...,165.00,1.00,1.00,1.00,1.00,0.00,0.00,1.00,1.00,2.00
75%,42.00,9.00,9302019.00,9.00,22.00,2019.00,1100.00,2019006895.00,2019006895.00,1.00,...,229.00,2.00,2.00,1.00,1.00,0.00,0.00,2.00,2.00,2.00
max,72.00,12.00,12312019.00,12.00,31.00,2020.00,1200.00,2019017419.00,2019017419.00,1.00,...,13204.00,9.00,9.00,1.00,1.00,2.00,2.00,9.00,9.00,9.00


In [40]:
###############################################################################################################################################################################
'''                                                                     Data Frame Shape                                                                                    '''
###############################################################################################################################################################################
df_19.shape

(418268, 250)

In [41]:
###############################################################################################################################################################################
'''                                                                    Data Frame Description                                                                               '''
###############################################################################################################################################################################
df_21.describe()

,_STATE,FMONTH,DISPCODE,_PSU,CTELENM1,PVTRESD1,COLGHOUS,STATERE1,LADULT1,COLGSEX,...,_FRTRES1,_VEGRES1,_FRUTSU1,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1
count,438693.00,438693.00,438693.00,438693.00,117786.00,117786.00,30.00,117786.00,117786.00,30.00,...,438693.00,438693.00,387606.00,378566.00,438693.00,438693.00,438693.00,438693.00,438693.00,438693.00
mean,30.74,6.41,1118.19,2021006064.89,1.00,1.00,1.00,1.00,1.01,1.63,...,0.88,0.86,178.34,271.54,2.27,2.26,0.99,0.99,0.13,0.15
std,15.33,3.42,38.58,6383.75,0.00,0.02,0.00,0.00,0.08,0.49,...,0.32,0.34,691.29,1036.23,2.49,2.71,0.07,0.09,0.35,0.38
min,1.00,1.00,1100.00,2021000001.00,1.00,1.00,1.00,1.00,1.00,1.00,...,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00
25%,20.00,3.00,1100.00,2021002091.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,57.00,114.00,1.00,1.00,1.00,1.00,0.00,0.00
50%,31.00,6.00,1100.00,2021004338.00,1.00,1.00,1.00,1.00,1.00,2.00,...,1.00,1.00,100.00,167.00,1.00,1.00,1.00,1.00,0.00,0.00
75%,41.00,9.00,1100.00,2021007674.00,1.00,1.00,1.00,1.00,1.00,2.00,...,1.00,1.00,200.00,229.00,2.00,2.00,1.00,1.00,0.00,0.00
max,78.00,12.00,1200.00,2021039095.00,2.00,2.00,1.00,1.00,2.00,2.00,...,1.00,1.00,19800.00,39600.00,9.00,9.00,1.00,1.00,2.00,2.00


In [42]:
###############################################################################################################################################################################
'''                                                                     Data Frame Shape                                                                                    '''
###############################################################################################################################################################################
df_21.shape

(438693, 250)

### <font color= lightblue> Step 3: Choose Feature Space

<font color=white>We will first need to choose our feature space.  The five demographics provided will be the first on the list:
<font color= red>
* Age:  _AGE_G
* Marital: MARITAL
* Sex: _SEX
* Income: INCOME2
* Education: _EDUCAG

<font color= white>
Next we want to choose some features which we think will have some bearing on the response columns. We also want to pair the 250 features down to 20 (as per professor). Another demographic could be their race and city vs rural living as well as access to health care (insurance) and routine checkups:
<br><br>
<font color= red>

* Race: _RACE
* Urban / Rural: _METSTAT
* Health Care Access (Insurance): PRIMINSR &ensp;&ensp;&ensp;&ensp;<font color=lightblue>-- Health Insurance data not included in the data provided<font color= red>
* Health Care Access (Routine checkup): CHECKUP1

<font color= white>
Also, we should also track physical activity, High blood pressure, High Cholesterol, smoking habits, drinking habits, BMI, kidney disease
<br><br>
<font color= red>

* Physical exersice: _TOTINDA
* HBP: _RFHYPE6 &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;<font color=lightblue>-- Blood Pressure metrics not included in the data provided<font color= white><br>

Instead we will use Currently taking blood pressure medication and Be very carefull with our clean up <br><font color= red>

* High Cholesterol: _RFCHOL3 &ensp;&ensp;&ensp;&ensp;<font color=lightblue>-- Cholesterol metrics not included in the data provided<font color= red>
* Four level smoker status: _SMOKER3
* Number of drinks per week: _DRNKWK1&ensp;&ensp;&ensp;&ensp;<font color=lightblue> Continuous Data<font color= red>
* Total Fruit per Day: _FRUTSU1&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;<font color=lightblue> Continuous Data<font color= red>
* Total Vegetables per Day: _VEGESU1&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;<font color=lightblue> Continuous Data<font color= red>
* Total French Fry per Day: FRNCHDA_&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;<font color=lightblue> Continuous Data<font color= red>
* Body Mass Index: _BMI5&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;<font color=lightblue> Continuous Data<font color= red>
* Kidney Disease: CHCKDNY2

<font color= white>
The last demographic that Tony would like to include specifically is whether the participant is a veteran:
br><br>
<font color= red>

* Veteran Status: VETERAN3


In [43]:
###############################################################################################################################################################################
'''                                                               Choosing Response and Features                                                                            '''
###############################################################################################################################################################################
df_19_trim = df_19.loc[:, ['ADDEPEV3', '_CASTHM1', 'CHCCOPD2', 'CHCSCNCR', 'CHCOCNCR', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3',
                            'DIABETE4', '_AGE_G', 'MARITAL', '_SEX', 'INCOME2', '_EDUCAG', '_RACE', '_METSTAT', 'CHECKUP1',
                            'BPMEDS', '_TOTINDA', '_SMOKER3', '_DRNKWK1', '_FRUTSU1', '_VEGESU1','FRNCHDA_', '_BMI5', 
                            'CHCKDNY2', 'VETERAN3']]
df_19_trim.shape

(418268, 27)

In [44]:
###############################################################################################################################################################################
'''                                                               Choosing Response and Features                                                                            '''
###############################################################################################################################################################################
df_21_trim = df_21.loc[:, ['ADDEPEV3', '_CASTHM1', 'CHCCOPD3', 'CHCSCNCR', 'CHCOCNCR', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3',
                            'DIABETE4', '_AGE_G', 'MARITAL', '_SEX', 'INCOME3', '_EDUCAG', '_RACE', '_METSTAT', 'CHECKUP1',
                            'BPMEDS', '_TOTINDA', '_SMOKER3', '_DRNKWK1', '_FRUTSU1', '_VEGESU1','FRNCHDA_', '_BMI5', 
                            'CHCKDNY2', 'VETERAN3']]
df_21_trim.shape

(438693, 27)

### <font color= lightblue> Step 4: Missing Data Analysis

##### BPMEDS
From looking at the data itself, most of the missing values from BPMEDS is because the previous question answered something other than yes to every choice other than yes.  Therefore we can infer that these were a no and that 7 and 9 should be dropped.
* 2021-
- missing: 266,560 
<br>vs<br>
- Previous other than yes: 266,560

there is only 0.26% which was I don't know or Refused

* 2019-
- missing: 248,634 
<br>vs<br>
- Previous other than yes: 248,634

there is only 0.21% which was I don't know or Refused

In [45]:
###############################################################################################################################################################################
'''                                                                   Analyze, Clean, and Impute                                                                            '''
###############################################################################################################################################################################
## Print Original Shape for comparisson
print('Original 19 shape:\t', df_21_trim.shape)
print('Original 21 shape:\t', df_21_trim.shape)

## Interpret and clean df
# output how many NaN there is per column
print('Missing values by field (Pre-Interpolate):')

# Visualize the finished Data Frame
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_19_trim.isnull().sum(), '\n')
    print(df_21_trim.isnull().sum(), '\n')

## Replace missing values in BPMEDS with No answer and drop 7 and 9 response as NAN
df_19_trim = df_19_trim[df_19_trim['BPMEDS'] != 7] 
df_19_trim = df_19_trim[df_19_trim['BPMEDS'] != 9]
df_19_trim['BPMEDS'].fillna(2, inplace= True) 

# How are we doing now
print(df_19_trim.isnull().sum(), '\n')
print(df_19_trim.shape)

# Drop the remaining NaN
df_19_trim.dropna(inplace= True)


## Replace missing values in BPMEDS with No answer and drop 7 and 9 response as NAN
df_21_trim = df_21_trim[df_21_trim['BPMEDS'] != 7] 
df_21_trim = df_21_trim[df_21_trim['BPMEDS'] != 9]
df_21_trim['BPMEDS'].fillna(2, inplace= True) 

# How are we doing now
print(df_21_trim.isnull().sum(), '\n')
print(df_21_trim.shape)

# Drop the remaining NaN
df_21_trim.dropna(inplace= True)

# Output final shape
print('Final 19 shape:\t', df_19_trim.shape)
print('Final 21 shape:\t', df_21_trim.shape)

Original 19 shape:	 (438693, 27)
Original 21 shape:	 (438693, 27)
Missing values by field (Pre-Interpolate):
ADDEPEV3        10
_CASTHM1         0
CHCCOPD2         8
CHCSCNCR         8
CHCOCNCR         9
CVDCRHD4         8
CVDINFR4        10
CVDSTRK3        11
DIABETE4         9
_AGE_G           0
MARITAL         49
_SEX             0
INCOME2       6881
_EDUCAG          0
_RACE            3
_METSTAT      8458
CHECKUP1        10
BPMEDS      248634
_TOTINDA         0
_SMOKER3         0
_DRNKWK1         0
_FRUTSU1     44600
_VEGESU1     53430
FRNCHDA_     38866
_BMI5        36203
CHCKDNY2        11
VETERAN3      1374
dtype: int64 

ADDEPEV3         3
_CASTHM1         0
CHCCOPD3         3
CHCSCNCR         2
CHCOCNCR         3
CVDCRHD4         2
CVDINFR4         2
CVDSTRK3         2
DIABETE4         3
_AGE_G           0
MARITAL          5
_SEX             0
INCOME3       8847
_EDUCAG          0
_RACE            0
_METSTAT      7054
CHECKUP1         2
BPMEDS      266560
_TOTINDA         0
_S

## Create categorical columns

- Column 1: Binary - indicates whether an individual has haf any of the chronic conditions. 
- Column 2: Multiclass - create a milticlass column with values that inidicate the total number of chronic conditions.

### <font color= lightblue> Step 5: Binary and Multiclass Response Created

<font color= yellow>First we create the binaries for 2019

In [46]:
###############################################################################################################################################################################
'''                                                                   Convert Response Column for 2019                                                                      '''
###############################################################################################################################################################################
## Binary Comorbidity Response Features from our respective responses
# Create the Depression comorbidity variable
df_19_trim['Depression'] = df_19_trim.apply(lambda row: 1 if row[0] == 1 else (0 if row[0] == 2 else np.NAN), axis=1)
print(df_19_trim[['Depression', 'ADDEPEV3']].value_counts())
print(df_19_trim['Depression'].isnull().sum())
print(df_19_trim['ADDEPEV3'].value_counts(), '\n')

# Create the Asthma comorbidity variable
df_19_trim['Asthma'] = df_19_trim.apply(lambda row: 1 if row[1] == 1 else (0 if row[1] == 2 else np.NAN), axis=1)
print(df_19_trim[['Asthma', '_CASTHM1']].value_counts())
print(df_19_trim['Asthma'].isnull().sum())
print(df_19_trim['_CASTHM1'].value_counts(), '\n')

# Create the COPD comorbidity variable
df_19_trim['COPD'] = df_19_trim.apply(lambda row: 1 if row[2] == 1 else (0 if row[2] == 2 else np.NAN), axis=1)
print(df_19_trim[['COPD', 'CHCCOPD2']].value_counts())
print(df_19_trim['COPD'].isnull().sum())
print(df_19_trim['CHCCOPD2'].value_counts(), '\n')

# Create the Cancer comorbidity variable
df_19_trim['Cancer'] = df_19_trim.apply(lambda row: 1 if row[3] == 1 else (1 if row[4] == 1 else (0 if row[3] == 2 else ( 0 if row[4] == 2 else np.NAN))), axis=1)
print(df_19_trim[['Cancer', 'CHCSCNCR', 'CHCOCNCR']].value_counts())
print(df_19_trim['Cancer'].isnull().sum())
print(df_19_trim['CHCSCNCR'].value_counts(), '\n')
print(df_19_trim['CHCOCNCR'].value_counts(), '\n')
print(df_19_trim['Cancer'].value_counts(), '\n')

# Create the Heart Condition comorbidity variable
df_19_trim['Heart'] = df_19_trim.apply(lambda row: 1 if row[5] == 1 else (1 if row[6] == 1 else (1 if row[7] == 1 else (0 if row[5] == 2 else (0 if row[6] == 2 else (0 if row[7] == 2 else np.NAN))))), axis=1)
print(df_19_trim[['Heart', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3']].value_counts())
print(df_19_trim['Heart'].isnull().sum())
print(df_19_trim['CVDCRHD4'].value_counts(), '\n')
print(df_19_trim['CVDINFR4'].value_counts(), '\n')
print(df_19_trim['CVDSTRK3'].value_counts(), '\n')
print(df_19_trim['Heart'].value_counts(), '\n')

# Create the Depression comorbidity variable
df_19_trim['Diabetes'] = df_19_trim.apply(lambda row: 1 if row[8] == 1 else (0 if row[8] == 2 else (0 if row[8] == 3 else (0 if row[8] == 4 else np.NAN))), axis=1)
print(df_19_trim[['Diabetes', 'DIABETE4']].value_counts())
print(df_19_trim['Diabetes'].isnull().sum())
print(df_19_trim['DIABETE4'].value_counts(), '\n')

print(df_19_trim.head())

Depression  ADDEPEV3
0.00        2.00        264512
1.00        1.00         64620
dtype: int64
1373
2.00    264512
1.00     64620
7.00      1188
9.00       185
Name: ADDEPEV3, dtype: int64 

Asthma  _CASTHM1
1.00    1           297376
0.00    2            30814
dtype: int64
2315
1    297376
2     30814
9      2315
Name: _CASTHM1, dtype: int64 

COPD  CHCCOPD2
0.00  2.00        300875
1.00  1.00         28179
dtype: int64
1451
2.00    300875
1.00     28179
7.00      1398
9.00        53
Name: CHCCOPD2, dtype: int64 

Cancer  CHCSCNCR  CHCOCNCR
0.00    2.00      2.00        267482
1.00    1.00      2.00         26841
        2.00      1.00         26565
        1.00      1.00          8057
0.00    7.00      2.00           660
        2.00      7.00           509
1.00    7.00      1.00           143
        1.00      7.00            89
0.00    2.00      9.00            58
        9.00      2.00            13
1.00    1.00      9.00            10
        9.00      1.00             1
dtype: 

<font color= yellow>Then we create the binaries for 2021

In [47]:
###############################################################################################################################################################################
'''                                                                   Convert Response Column for 2021                                                                      '''
###############################################################################################################################################################################
## Binary Comorbidity Response Features from our respective responses
# Create the Depression comorbidity variable
df_21_trim['Depression'] = df_21_trim.apply(lambda row: 1 if row[0] == 1 else (0 if row[0] == 2 else np.NAN), axis=1)
print(df_21_trim[['Depression', 'ADDEPEV3']].value_counts())
print(df_21_trim['Depression'].isnull().sum())
print(df_21_trim['ADDEPEV3'].value_counts(), '\n')

# Create the Asthma comorbidity variable
df_21_trim['Asthma'] = df_21_trim.apply(lambda row: 1 if row[1] == 1 else (0 if row[1] == 2 else np.NAN), axis=1)
print(df_21_trim[['Asthma', '_CASTHM1']].value_counts())
print(df_21_trim['Asthma'].isnull().sum())
print(df_21_trim['_CASTHM1'].value_counts(), '\n')

# Create the COPD comorbidity variable
df_21_trim['COPD'] = df_21_trim.apply(lambda row: 1 if row[2] == 1 else (0 if row[2] == 2 else np.NAN), axis=1)
print(df_21_trim[['COPD', 'CHCCOPD3']].value_counts())
print(df_21_trim['COPD'].isnull().sum())
print(df_21_trim['CHCCOPD3'].value_counts(), '\n')

# Create the Cancer comorbidity variable
df_21_trim['Cancer'] = df_21_trim.apply(lambda row: 1 if row[3] == 1 else (1 if row[4] == 1 else (0 if row[3] == 2 else ( 0 if row[4] == 2 else np.NAN))), axis=1)
print(df_21_trim[['Cancer', 'CHCSCNCR', 'CHCOCNCR']].value_counts())
print(df_21_trim['Cancer'].isnull().sum())
print(df_21_trim['CHCSCNCR'].value_counts(), '\n')
print(df_21_trim['CHCOCNCR'].value_counts(), '\n')
print(df_21_trim['Cancer'].value_counts(), '\n')

# Create the Heart Condition comorbidity variable
df_21_trim['Heart'] = df_21_trim.apply(lambda row: 1 if row[5] == 1 else (1 if row[6] == 1 else (1 if row[7] == 1 else (0 if row[5] == 2 else (0 if row[6] == 2 else (0 if row[7] == 2 else np.NAN))))), axis=1)
print(df_21_trim[['Heart', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3']].value_counts())
print(df_21_trim['Heart'].isnull().sum())
print(df_21_trim['CVDCRHD4'].value_counts(), '\n')
print(df_21_trim['CVDINFR4'].value_counts(), '\n')
print(df_21_trim['CVDSTRK3'].value_counts(), '\n')
print(df_21_trim['Heart'].value_counts(), '\n')

# Create the Depression comorbidity variable
df_21_trim['Diabetes'] = df_21_trim.apply(lambda row: 1 if row[8] == 1 else (0 if row[8] == 2 else (0 if row[8] == 3 else (0 if row[8] == 4 else np.NAN))), axis=1)
print(df_21_trim[['Diabetes', 'DIABETE4']].value_counts())
print(df_21_trim['Diabetes'].isnull().sum())
print(df_21_trim['DIABETE4'].value_counts(), '\n')

print(df_21_trim.head())

Depression  ADDEPEV3
0.00        2.00        268786
1.00        1.00         68765
dtype: int64
1447
2.00    268786
1.00     68765
7.00      1183
9.00       264
Name: ADDEPEV3, dtype: int64 

Asthma  _CASTHM1
1.00    1.00        303552
0.00    2.00         33089
dtype: int64
2357
1.00    303552
2.00     33089
9.00      2357
Name: _CASTHM1, dtype: int64 

COPD  CHCCOPD3
0.00  2.00        311016
1.00  1.00         26759
dtype: int64
1223
2.00    311016
1.00     26759
7.00      1157
9.00        66
Name: CHCCOPD3, dtype: int64 

Cancer  CHCSCNCR  CHCOCNCR
0.00    2.00      2.00        278474
1.00    2.00      1.00         25632
        1.00      2.00         25540
                  1.00          7869
0.00    7.00      2.00           588
        2.00      7.00           468
1.00    7.00      1.00           137
        1.00      7.00            88
0.00    2.00      9.00            56
        9.00      2.00            17
1.00    1.00      9.00            14
        9.00      1.00             

<font color= yellow>Drop the coloumns no longer needed and any missing values introduced

In [48]:
###############################################################################################################################################################################
'''                                                                   Re-Clean Data fropm both years                                                                        '''
###############################################################################################################################################################################
## First we need to drop the columns we no longer need and then the new NaNs for 2019
print('df_19_trim Original Shape_\t', df_19_trim.shape)
df_19_trim.drop(columns=['ADDEPEV3', '_CASTHM1', 'CHCCOPD2', 'CHCSCNCR', 'CHCOCNCR', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3', 'DIABETE4'], inplace=True)
df_19_trim.dropna(inplace= True)
print('df_19_trim New Shape_\t', df_19_trim.shape, '\n')

## First we need to drop the columns we no longer need and then the new NaNs for 2021
print('df_21_trim Original Shape_\t', df_21_trim.shape)
df_21_trim.drop(columns=['ADDEPEV3', '_CASTHM1', 'CHCCOPD3', 'CHCSCNCR', 'CHCOCNCR', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3', 'DIABETE4'], inplace=True)
df_21_trim.dropna(inplace= True)
print('df_19_trim New Shape_\t', df_21_trim.shape)

df_19_trim Original Shape_	 (330505, 33)
df_19_trim New Shape_	 (325245, 24) 

df_21_trim Original Shape_	 (338998, 33)
df_19_trim New Shape_	 (333899, 24)


<font color= yellow>Now we can move on to our last Comorbidity Variable for each year

In [50]:
###############################################################################################################################################################################
'''                                                                   Convert Response Column for 2019/2021                                                                 '''
###############################################################################################################################################################################
## Multiclass Response from the Binary Comorbidities
# First for 2019
df_19_trim['Comorbidity'] = df_19_trim.apply(lambda row: 1 if row[18] == 1 else (1 if row[19] == 1 else (1 if row[20] == 1 else (1 if row[21] == 1 else (1 if row[22] == 1 else (1 if row[23] == 1 else 0))))), axis=1)

# Then for 2021
df_21_trim['Comorbidity'] = df_21_trim.apply(lambda row: 1 if row[18] == 1 else (1 if row[19] == 1 else (1 if row[20] == 1 else (1 if row[21] == 1 else (1 if row[22] == 1 else (1 if row[23] == 1 else 0))))), axis=1)

### <font color= lightblue> Step 6: Make sure there are no NaNs left over

In [54]:
###############################################################################################################################################################################
'''                                                            Check for missing values again for 2019/2021                                                                 '''
###############################################################################################################################################################################
# output how many NaN there is per column
print('Missing values by field (Pre-Interpolate):')

# Visualize the finished Data Frame
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_19_trim.isnull().sum(), '\n')
    print(df_21_trim.isnull().sum(), '\n')

# We can see that there is one column left in each that have a different name so let us fix this now
df_21_trim = df_21_trim.rename(columns={'INCOME3': 'INCOME2'})

# Recheck by visualizing the finished Data Frame
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_19_trim.isnull().sum(), '\n')
    print(df_21_trim.isnull().sum(), '\n')

# Final shape of DF's
print('Final Shapes:\n', df_19_trim.shape, '\n', df_21_trim.shape)

Missing values by field (Pre-Interpolate):
_AGE_G         0
MARITAL        0
_SEX           0
INCOME2        0
_EDUCAG        0
_RACE          0
_METSTAT       0
CHECKUP1       0
BPMEDS         0
_TOTINDA       0
_SMOKER3       0
_DRNKWK1       0
_FRUTSU1       0
_VEGESU1       0
FRNCHDA_       0
_BMI5          0
CHCKDNY2       0
VETERAN3       0
Depression     0
Asthma         0
COPD           0
Cancer         0
Heart          0
Diabetes       0
Comorbidity    0
dtype: int64 

_AGE_G         0
MARITAL        0
_SEX           0
INCOME3        0
_EDUCAG        0
_RACE          0
_METSTAT       0
CHECKUP1       0
BPMEDS         0
_TOTINDA       0
_SMOKER3       0
_DRNKWK1       0
_FRUTSU1       0
_VEGESU1       0
FRNCHDA_       0
_BMI5          0
CHCKDNY2       0
VETERAN3       0
Depression     0
Asthma         0
COPD           0
Cancer         0
Heart          0
Diabetes       0
Comorbidity    0
dtype: int64 

_AGE_G         0
MARITAL        0
_SEX           0
INCOME2        0
_EDUCAG  

### <font color= lightblue> Step 6: Merge Data Frames

In [55]:
df = pd.concat([df_19_trim, df_21_trim])
df.head()

,_AGE_G,MARITAL,_SEX,INCOME2,_EDUCAG,_RACE,_METSTAT,CHECKUP1,BPMEDS,_TOTINDA,...,_BMI5,CHCKDNY2,VETERAN3,Depression,Asthma,COPD,Cancer,Heart,Diabetes,Comorbidity
0,6.00,2.00,2.00,3.00,1.00,2.00,1.00,1.00,1.00,2.00,...,2817.00,2.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1
1,6.00,1.00,2.00,5.00,3.00,1.00,1.00,1.00,2.00,1.00,...,1854.00,2.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1
2,6.00,3.00,2.00,7.00,4.00,2.00,1.00,1.00,1.00,1.00,...,3162.00,2.00,2.00,0.00,1.00,0.00,0.00,0.00,1.00,1
4,6.00,1.00,2.00,99.00,3.00,1.00,2.00,1.00,2.00,2.00,...,2148.00,2.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1
6,6.00,2.00,1.00,7.00,4.00,1.00,2.00,1.00,2.00,1.00,...,3298.00,2.00,2.00,0.00,0.00,1.00,0.00,0.00,1.00,1


In [56]:
df.describe()

,_AGE_G,MARITAL,_SEX,INCOME2,_EDUCAG,_RACE,_METSTAT,CHECKUP1,BPMEDS,_TOTINDA,...,_BMI5,CHCKDNY2,VETERAN3,Depression,Asthma,COPD,Cancer,Heart,Diabetes,Comorbidity
count,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,...,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00,659144.00
mean,4.42,2.31,1.53,18.67,3.06,1.97,1.31,1.43,1.67,1.25,...,2847.10,1.97,1.88,0.20,0.90,0.08,0.18,0.11,0.13,0.97
std,1.59,1.72,0.50,29.69,0.97,2.21,0.46,1.04,0.47,0.51,...,647.48,0.33,0.41,0.40,0.29,0.27,0.38,0.32,0.34,0.18
min,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1200.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,3.00,1.00,1.00,5.00,2.00,1.00,1.00,1.00,1.00,1.00,...,2412.00,2.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
50%,5.00,1.00,2.00,7.00,3.00,1.00,1.00,1.00,2.00,1.00,...,2741.00,2.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
75%,6.00,3.00,2.00,9.00,4.00,1.00,2.00,1.00,2.00,1.00,...,3162.00,2.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
max,6.00,9.00,2.00,99.00,9.00,9.00,2.00,9.00,2.00,9.00,...,9933.00,9.00,9.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [57]:
df.shape

(659144, 25)

<font color= white>The final Data Frames have the following Response Columns<font color= red>

* Depression
* Asthma
* COPD
* Cancer
* Heart
* Diabetes
* Comorbidity

<font color=white>And the following Feature Space<font color= red>

* _AGE_G
* MARITAL
* _SEX
* INCOME2
* _EDUCAG
* _RACE
* _METSTAT
* CHECKUP1
* BPMEDS
* _TOTINDA
* _SMOKER3
* _DRNKWK1
* _FRUTSU1
* _VEGESU1
* FRNCHDA_
* _BMI5
* CHCKDNY2
* VETERAN3

<font color=white>The shapes of the different Dat Frames are as follows:<font color= red>

1. 2019: <br>
325245, 25<br><br>
2. 2021: <br>
333899, 25<br><br>
3. Merged: <br>
659144, 25<br><br>

## Analysis 1
Using both years, run exploratory data analysis using crosstabs, visuals, and basic frequency distributions to understand how chronic conditions are distribituted across geography and demography. You may also use the newly created comorbidity variables for this analysis. 

- Summarize salient features of the healthiest and least healthy states in the country.
- Discuss if you noticed any associations of the risk factors such as Age, Sex, Income, Education, Marital Status etc. with the level of comorbidities, while comparing the two years of data.

## Analysis 2
1. Using only 2021, use several classification algorithms to classify comorbility variables 1 and 2:
- Logistic Regression
- KNN
- RF
- Gradient Boosting
- XGBoost
- Catboost

2. Write a short report describing the performance metrics. 
3. In the end, choose one model each for the classification of both categorical variables

### Logistic Regresison

### KNN

### Random Forest

### Gradient Boost

### XGBoost

### Catboost